In [1]:
import numpy as np
import skimage.io
from scipy import ndimage
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim
import os
import matplotlib.pyplot as plt

In [2]:
def median_filter_output(gt_path, noisy_path, noise_factor, kernel_size, result_path):
        
    nf = str(noise_factor).replace(".", "")
    psnr_total  = 0
    ssim_total  = 0
    
    for root, dirs, files in os.walk(gt_path, topdown=False):
        for name in files:
            img = skimage.io.imread(gt_path + name)
            
            noisy_img = skimage.io.imread(noisy_path + nf + '/' + nf + '_' + name)
            noisy_img_normalized = noisy_img/255.0
            out = ndimage.median_filter(noisy_img_normalized, kernel_size)
            out = np.clip(out, 0,1)*255
            out = out.astype('uint8')
            
            psnr_total += compare_psnr(img, out)
            ssim_total += compare_ssim(img, out, channel_axis=2)
            
            skimage.io.imsave(result_path + 'median_' + str(kernel_size) + '_' + name, out)
        
    psnr_total /= len(files)
    ssim_total /= len(files)
    
    print("PSNR: ", psnr_total)
    print("SSIM: ", ssim_total)
    
    return psnr_total, ssim_total

In [3]:
noise_factor = 0.3
nf = str(noise_factor).replace(".", "")
kernel_size = 3

In [4]:
mri_test_path = './data/mri_images_gt/bt_gt_test/'
ct_path = './data/ct_images_gt/'
lympho_path = './data/lymphocytes_images_gt/'

mri_test_path_noisy = mri_test_path + '../bt_noisy_test/'
ct_path_noisy = ct_path + '../ct_noisy/'
lympho_path_noisy = lympho_path + '../lymphocytes_noisy/'

result_path_mri = './data/results/median/mri_test/' + nf + '/'
result_path_ct = './data/results/median/ct_test/'  + nf + '/'
result_path_lympho = './data/results/median/lympho_test/' + nf + '/'

os.makedirs(result_path_mri, exist_ok=True)
os.makedirs(result_path_ct, exist_ok=True)
os.makedirs(result_path_lympho, exist_ok=True)

In [5]:
psnr, ssim = median_filter_output(mri_test_path, mri_test_path_noisy, noise_factor, kernel_size, result_path_mri)

PSNR:  20.241053079618776
SSIM:  0.2399206214675543


In [6]:
psnr, ssim = median_filter_output(ct_path, ct_path_noisy, noise_factor, kernel_size, result_path_ct)

PSNR:  20.625862021647038
SSIM:  0.21566251955653323


In [7]:
psnr, ssim = median_filter_output(lympho_path, lympho_path_noisy, noise_factor, kernel_size, result_path_lympho)

PSNR:  17.078347169892528
SSIM:  0.6041632442330819
